## 🧰 Initial Settings

### 📦 Import Standard Libraries

In [1]:
import os, glob, shutil, random

### 📥 Download Dataset from Kaggle

In [2]:
from kaggle.api.kaggle_api_extended import KaggleApi

os.environ["KAGGLE_CONFIG_DIR"] = os.getcwd()

api = KaggleApi()
api.authenticate()

DATASET_SLUG = "hhshamimkhan/face-emotion-datasets"
api.dataset_download_files(DATASET_SLUG, unzip=True)

Dataset URL: https://www.kaggle.com/datasets/hhshamimkhan/face-emotion-datasets


## 🛠️ Dataset Preparation for YOLO

### ⚙️ Dataset Paths, Split Ratios, and Configuration Settings

In [3]:
SRC_ROOT = "Face emotion datasets"
OUT_ROOT = "dataset_yolo"
R_TRAIN, R_VAL = 0.70, 0.15
MAX_PER_CLASS_TOTAL = 400
SEED = 42
VALID_EXT = (".jpg", ".jpeg", ".png", ".bmp", ".webp")

random.seed(SEED)

### 📊 Count Images per Class and Total Samples

In [4]:
counts = {}
for cls in sorted(d for d in os.listdir(SRC_ROOT) if os.path.isdir(os.path.join(SRC_ROOT,d))):
    n = len([p for p in glob.glob(os.path.join(SRC_ROOT, cls, "*")) if p.lower().endswith(VALID_EXT)])
    counts[cls] = n

for k,v in counts.items():
    print(f"{k}: {v}")
print("Total:", sum(counts.values()))

Anger: 2310
Content: 772
Disgust: 788
Fear: 906
Happy: 4929
Neutral: 5016
Sad: 4771
SillyFace: 1116
Surprise: 2231
Total: 22839


### 🔎 Discover Classes and Assign IDs

In [5]:
# 1) Descubrir clases (orden alfabético estable)
classes = sorted(
    [d for d in os.listdir(SRC_ROOT) if os.path.isdir(os.path.join(SRC_ROOT, d))]
)
name2id = {c: i for i, c in enumerate(classes)}
print("Clases -> IDs:")
name2id

Clases -> IDs:


{'Anger': 0,
 'Content': 1,
 'Disgust': 2,
 'Fear': 3,
 'Happy': 4,
 'Neutral': 5,
 'Sad': 6,
 'SillyFace': 7,
 'Surprise': 8}

### 🗂️ Split Dataset into Train/Val/Test and Generate YOLO Labels

In [6]:
for s in ["train", "val", "test"]:
    os.makedirs(os.path.join(OUT_ROOT, "images", s), exist_ok=True)
    os.makedirs(os.path.join(OUT_ROOT, "labels", s), exist_ok=True)

for cls in classes:
    imgs = [
        p
        for p in glob.glob(os.path.join(SRC_ROOT, cls, "*"))
        if p.lower().endswith(VALID_EXT)
    ]

    random.shuffle(imgs)
   
    imgs = imgs[:min(len(imgs), MAX_PER_CLASS_TOTAL)]

    n = len(imgs)
    n_tr = int(n * R_TRAIN)
    n_va = int(n * R_VAL)
    n_te = n - n_tr - n_va

    splits = [
        ("train", imgs[:n_tr]),
        ("val", imgs[n_tr : n_tr + n_va]),
        ("test", imgs[n_tr + n_va :]),
    ]

    for split, items in splits:
        for src_img in items:

            fn = os.path.basename(src_img)
            shutil.copy2(src_img, os.path.join(OUT_ROOT, "images", split, fn))

            with open(
                os.path.join(
                    OUT_ROOT, "labels", split, os.path.splitext(fn)[0] + ".txt"
                ),
                "w",
            ) as f:
                f.write(f"{name2id[cls]} 0.5 0.5 1.0 1.0\n")

In [7]:
for split in ["train","val","test"]:
    img_dir = os.path.join("dataset_yolo","images",split)
    n = len([f for f in os.listdir(img_dir) if f.lower().endswith((".jpg",".jpeg",".png",".bmp",".webp"))])
    print(split, "→", n, "images")


train → 2520 images
val → 540 images
test → 540 images


### 📄 Create YOLO Configuration File (data.yaml)

In [8]:
with open(os.path.join(OUT_ROOT, "data.yaml"), "w", encoding="utf-8") as f:
    f.write(
        f"path: {OUT_ROOT}\ntrain: images/train\nval: images/val\n"
        f"test: images/test\nnames: [{', '.join(classes)}]\n"
    )

## 🤖 Train YOLO Model

### 📦 Load Pre-trained YOLO Model

In [9]:
from ultralytics import YOLO

yolo = YOLO("yolo11n.pt")

### 🚀 Training YOLO Model on Dataset

In [10]:
yolo.train(
    data="dataset_yolo/data.yaml",
    epochs=40,
    imgsz=640,
    batch=16,
    patience=10,
)

New https://pypi.org/project/ultralytics/8.3.202 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.201 🚀 Python-3.12.3 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset_yolo/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, op

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f4305d13500>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.0470